# DTOcean Installation Module Test

In [1]:
%matplotlib inline

In [2]:
from IPython.display import display, HTML

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (14.0, 8.0)

In [4]:
import numpy as np

In [5]:
from dtocean_core import start_logging
from dtocean_core.core import Core
from dtocean_core.menu import ModuleMenu, ProjectMenu, ThemeMenu
from dtocean_core.pipeline import Tree

In [6]:
def html_list(x):
    message = "<ul>"
    for name in x:
        message += "<li>{}</li>".format(name)
    message += "</ul>"
    return message
def html_dict(x):
    message = "<ul>"
    for name, status in x.iteritems():
        message += "<li>{}: <b>{}</b></li>".format(name, status)
    message += "</ul>"
    return message

In [7]:
# Bring up the logger
start_logging()

2017-05-11 11:16:54,243 - INFO - dtocean_core - Begin logging for dtocean_core.


## Create the core, menus and pipeline tree

The core object carrys all the system information and is operated on by the other classes

In [8]:
new_core = Core()
project_menu = ProjectMenu()
module_menu = ModuleMenu()
theme_menu = ThemeMenu()
pipe_tree = Tree()

## Create a new project

In [9]:
project_title = "DTOcean"  
new_project = project_menu.new_project(new_core, project_title)

2017-05-11 11:16:58,851 - INFO - aneris.entity.simulation - Created new Simulation with title "Default"
2017-05-11 11:16:58,855 - INFO - aneris.control.simulation - Datastate with level "initial" stored
2017-05-11 11:16:58,861 - INFO - aneris.control.pipeline - New Hub created for interface ProjectInterface.


## Set the device type

In [10]:
options_branch = pipe_tree.get_branch(new_core, new_project, "System Type Selection")
variable_id = "device.system_type"
my_var = options_branch.get_input_variable(new_core, new_project, variable_id)
my_var.set_raw_interface(new_core, "Wave Floating")
my_var.read(new_core, new_project)

2017-05-11 11:16:58,887 - INFO - aneris.control.data - New "device.system_type" data stored with index 8HJI6R
2017-05-11 11:16:58,890 - INFO - aneris.control.simulation - Datastate stored


## Initiate the pipeline
This step will be important when the database is incorporated into the system as it will effect the operation of the pipeline.

In [11]:
project_menu.initiate_pipeline(new_core, new_project)

2017-05-11 11:16:58,908 - INFO - aneris.control.simulation - Datastate with level "system type selection start" stored
2017-05-11 11:16:58,936 - INFO - aneris.control.data - New "hidden.pipeline_active" data stored with index I584LO
2017-05-11 11:16:58,937 - INFO - aneris.control.simulation - Datastate with level "system type selection output" stored
2017-05-11 11:16:58,941 - INFO - aneris.control.pipeline - New Pipeline created for interface ModuleInterface.
2017-05-11 11:16:58,944 - INFO - aneris.control.pipeline - New Hub created for interface ThemeInterface.


## Discover available modules

In [12]:
names = module_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

## Activate a module

Note that the order of activation is important and that we can't deactivate yet!

In [13]:
module_name = 'Installation'
module_menu.activate(new_core, new_project, module_name)

## Activate the Economics themes

In [14]:
names = theme_menu.get_available(new_core, new_project)
message = html_list(names)
HTML(message)

In [15]:
theme_menu.activate(new_core, new_project, "Economics")

## Check the status of the module inputs

In [16]:
installation_branch = pipe_tree.get_branch(new_core, new_project, 'Installation')
input_status = installation_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Initiate the dataflow
This indicates that the filtering and module / theme selections are complete

In [17]:
project_menu.initiate_dataflow(new_core, new_project)

2017-05-11 11:16:59,088 - INFO - aneris.control.data - New "hidden.dataflow_active" data stored with index W9MJ0K
2017-05-11 11:16:59,089 - INFO - aneris.control.simulation - Datastate stored
2017-05-11 11:16:59,101 - INFO - aneris.control.simulation - Datastate with level "modules initial" stored


## Load test data

Prepare the test data for loading. The files required can be found in the test_data directory of the source code and should be copied to the directory that the notebook is running. When the python file is run a pickle file is generated containing a dictionary of inputs.

In [18]:
%run inputs_wp5.py

generate test data: C:\Users\mtopper\Programming\Python\notebooks\inputs_wp5.pkl


In [19]:
installation_branch.read_test_data(new_core,
                            new_project,
                            "inputs_wp5.pkl")

2017-05-11 11:17:20,489 - INFO - aneris.control.data - New "component.vehicle_vessel_clv" data stored with index 3ZM2RZ
2017-05-11 11:17:20,489 - INFO - aneris.control.data - New "project.split_pipe_safety_factors" data stored with index O3KGQZ
2017-05-11 11:17:20,489 - INFO - aneris.control.data - New "project.landfall_contruction_technique" data stored with index 7J9FC8
2017-05-11 11:17:20,489 - INFO - aneris.control.data - New "device.bollard_pull" data stored with index 4Y93LJ
2017-05-11 11:17:20,505 - INFO - aneris.control.data - New "device.transportation_method" data stored with index ZNA9S5
2017-05-11 11:17:20,505 - INFO - aneris.control.data - New "project.loading_rate" data stored with index RWEZ7U
2017-05-11 11:17:20,505 - INFO - aneris.control.data - New "project.port_percentage_cost" data stored with index CLSSX8
2017-05-11 11:17:20,505 - INFO - aneris.control.data - New "project.foundations_component_data" data stored with index D949DN
2017-05-11 11:17:21,197 - INFO - ane

## Prepare the Economics Theme

In [20]:
theme_name = "Economics"
eco_branch = pipe_tree.get_branch(new_core, new_project, "Economics")

input_status = eco_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Set the discount rate and cost estimates and project lifetime

In [21]:
discount_rate = 0.05
device_cost = 1000000
project_lifetime = 20

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "project.discount_rate")
new_var.set_raw_interface(new_core, discount_rate)
new_var.read(new_core, new_project)

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "device.system_cost")
new_var.set_raw_interface(new_core, device_cost)
new_var.read(new_core, new_project)

new_var = eco_branch.get_input_variable(new_core, new_project,
                                        "project.lifetime")
new_var.set_raw_interface(new_core, project_lifetime)
new_var.read(new_core, new_project)

2017-05-11 11:17:22,683 - INFO - aneris.control.data - New "project.discount_rate" data stored with index IXVJ4S
2017-05-11 11:17:22,683 - INFO - aneris.control.simulation - Datastate stored
2017-05-11 11:17:22,713 - INFO - aneris.control.data - New "device.system_cost" data stored with index 3Z1BKO
2017-05-11 11:17:22,713 - INFO - aneris.control.simulation - Datastate stored
2017-05-11 11:17:22,744 - INFO - aneris.control.data - New "project.lifetime" data stored with index 3RV30Z
2017-05-11 11:17:22,744 - INFO - aneris.control.simulation - Datastate stored


## Check if the module can be executed

In [22]:
can_execute = module_menu.is_executable(new_core, new_project, module_name)
display(can_execute)

True

In [23]:
input_status = installation_branch.get_input_status(new_core, new_project)
message = html_dict(input_status)
HTML(message)

## Execute the current module

The "current" module refers to the next module to be executed in the chain (pipeline) of modules. This command will only execute that module and another will be used for executing all of the modules at once.

Note, any data supplied by the module will be automatically copied into the active data state.

In [24]:
module_menu.execute_current(new_core, new_project)

2017-05-11 11:17:22,822 - INFO - aneris.control.simulation - Datastate with level "installation start" stored
2017-05-11 11:17:23,683 - INFO - dtocean_installation.main - Installation port selected...
2017-05-11 11:17:23,792 - WARNING - dtocean_logistics.phases.install.Driven - Drilling technique inadequate for site soil types.
2017-05-11 11:17:23,822 - INFO - dtocean_installation.main - Checking installation requirements for phase: Installation of driven piles anchors/foundations.
2017-05-11 11:17:23,933 - INFO - dtocean_installation.main - 7 possible solutions found.
2017-05-11 11:17:24,598 - INFO - dtocean_installation.main - Final solution found.
2017-05-11 11:17:24,614 - INFO - dtocean_installation.main - Checking installation requirements for phase: Installation of mooring systems with pile anchors.
2017-05-11 11:17:24,707 - INFO - dtocean_installation.main - 5 possible solutions found.
2017-05-11 11:17:25,036 - INFO - dtocean_installation.main - Final solution found.
2017-05-11 

## Examine the results

In [25]:
output_status = installation_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [27]:
economics_data = new_core.get_data_value(new_project, "project.device_phase_installation_costs")
economics_data

,Equipment,Port,Vessel
Component,,,
Device,268591.844184,618392.379649,5.296940e+06


In [28]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.device_phase_installation_cost_breakdown")
economics_data_breakdown

{'Device': 6183923.796485089}

In [30]:
port = new_core.get_data_value(new_project, "project.port")
port

'Portland port'

In [31]:
comp_cost = new_core.get_data_value(new_project, "project.electrical_phase_installation_costs")
comp_cost

,Equipment,Port,Vessel
Component,,,
Dynamic Cables,4.623383e+04,9.763611e+04,8.324911e+05
Export Cables,2.976529e+06,2.420907e+06,1.881163e+07
Inter-Array Cables,1.792494e+05,1.548217e+05,1.214146e+06
Collection Points,3.959992e+04,5.079692e+04,4.175723e+05


In [32]:
comp_time = new_core.get_data_value(new_project, "project.electrical_phase_installation_times")
comp_time

,Preparation,Operations,Transit,Waiting
Component,,,,
Dynamic Cables,50.004444,17.549848,177.118894,0.000000
Export Cables,49.002222,9.946996,177.193148,4724.285714
Inter-Array Cables,50.004444,9.380188,177.104906,109.285714
Collection Points,97.000000,14.320314,138.130497,14.285714


In [33]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.electrical_phase_installation_time_breakdown")
economics_data_breakdown

{'Collection Points': 263.7365251068964,
 'Dynamic Cables': 244.67318677068744,
 'Export Cables': 4960.428080599214,
 'Inter-Array Cables': 345.77525203803776}

In [34]:
comp_cost = new_core.get_data_value(new_project, "project.mooring_phase_installation_costs")
comp_cost

,Equipment,Port,Vessel
Component,,,
Driven Piles,120524.773243,85369.075152,647796.903125
Pile Anchors,28640.068624,13789.662671,95466.895412


In [35]:
comp_time = new_core.get_data_value(new_project, "project.mooring_phase_installation_times")
comp_time

,Preparation,Operations,Transit,Waiting
Component,,,,
Driven Piles,78.0,42.048516,169.089773,0.0
Pile Anchors,78.0,12.000000,108.589763,0.0


In [36]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.mooring_phase_installation_time_breakdown")
economics_data_breakdown

{'Driven Piles': 289.1382888932672, 'Pile Anchors': 198.58976262629199}

In [37]:
device_cost_breakdown = new_core.get_data_value(new_project, "project.device_phase_cost_class_breakdown")
electrical_cost_breakdown = new_core.get_data_value(new_project, "project.electrical_phase_installation_cost_breakdown")
mooring_cost_breakdown = new_core.get_data_value(new_project, "project.mooring_phase_installation_cost_breakdown")

In [38]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_phase_cost_breakdown")
economics_data_breakdown

{'Contingency': 3441712.9045441514,
 'Devices': 6183923.796485089,
 'Electrical Sub-Systems': 27241617.87072934,
 'Mooring and Foundations': 991587.3782270891}

In [39]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_cost_class_breakdown")
economics_data_breakdown

{'Contingency': 3441712.9045441514,
 'Equipment': 3659368.5831726454,
 'Port': 3441712.9045441523,
 'Vessel': 27316047.55772472}

In [40]:
device_time_breakdown = new_core.get_data_value(new_project, "project.device_phase_time_class_breakdown")
device_time_breakdown

{'Operations': 14.0,
 'Preparation': 74.0,
 'Transit': 127.77017100184622,
 'Waiting': 989.1428571428571}

In [41]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_phase_time_breakdown")
economics_data_breakdown

{'Devices': 1204.9130281447033,
 'Electrical Sub-Systems': 5814.6130445148365,
 'Mooring and Foundations': 487.7280515195592}

In [42]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_time_class_breakdown")
economics_data_breakdown

{'Operations': 119.24586136347982,
 'Preparation': 476.0111111111111,
 'Transit': 1074.997151704507,
 'Waiting': 5837.000000000001}

In [43]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.installation_economics_data")
economics_data_breakdown

,Key Identifier,Cost,Quantity,Year
0,Contingency,3.441713e+06,1,0
1,Mooring and Foundations,9.915874e+05,1,0
2,Devices,6.183924e+06,1,0
3,Electrical Sub-Systems,2.724162e+07,1,0


In [44]:
output_status = eco_branch.get_output_status(new_core, new_project)
message = html_dict(output_status)
HTML(message)

In [45]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.capex_breakdown")
economics_data_breakdown

{'Installation': 37858841.94998567}

In [46]:
economics_data_breakdown = new_core.get_data_value(new_project, "project.capex_total")
economics_data_breakdown

37858841.94998567